This is full implementation of Yimin Nie's solution as given in [discussion](https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/88950)

I wanted to get target encoding working without leaking the target variable. The above post by Yimin Nie gives the code to do that.
I packaged it as a complete solution in a kernel using the starter kernel [30 lines Starter Solution #FAST](https://www.kaggle.com/jesucristo/30-lines-starter-solution-fast)

Hope this serves as a reference for anyone who want to know how to get target encoding working. If there is something wrong please comment.

Changes: 
Removed DF reversals as it doesn't help in target encoding narration. 
feature_fraction needs to be set to 1.

This does not use real/fake data split. Like they have shown in other kernels we can break 0.901 without exploiting the test data split.

Caution: This kernel takes several hours to run.


All credits to go to [Yimin Nie](https://www.kaggle.com/chikenfeet) for posting the code. Upvote the [original discussion](https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/88950) if you liked this.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm_notebook as tqdm

import os
print(os.listdir("../input"))


['sample_submission.csv', 'test.csv', 'train.csv']


In [2]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
features = [c for c in train_df.columns if c not in ['ID_code', 'target']] #basic features
target = train_df['target']

In [3]:
def t_encoding(tr,va,by):
    df = tr.groupby(by).agg({'target':['sum','count']})
    cols = ['sum_y','count_y']
    df.columns = cols
    df = df.reset_index()
    df = df.sort_values(by)

    df['r'] = df['sum_y'].cumsum() / df['count_y'].cumsum()  # I think this is the key operation
    df.drop(['sum_y','count_y'],axis=1,inplace=True)
    return va.merge(df,on=by,how='left')['r'].values


In [4]:

var_cols = features

for col in tqdm(var_cols):
    te_r = t_encoding(train_df,test_df,col) 
    test_df.loc[:,col+'_r'] = te_r


folds = StratifiedKFold(n_splits=12, shuffle=False, random_state=99999)
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    
    tr = train_df.loc[trn_idx,var_cols+['target']]
    va = train_df.loc[val_idx,var_cols+['target']]

    for col in tqdm(var_cols):
        encoded = t_encoding(tr,va,col)
        train_df.loc[val_idx,col+'_r'] = encoded
                
    
features = [col for col in train_df.columns if ('var' in col)]

train_df = train_df[features].reset_index(drop=True)
print('data prepared: {}'.format(train_df.shape))


Fold 0



Fold 1



Fold 2



Fold 3



Fold 4


...


Fold 8



Fold 9



Fold 10



Fold 11



data prepared: (200000, 400)


In [5]:
param = {
    'bagging_freq': 5,          
    'bagging_fraction': 0.38,   'boost_from_average':'false',   
    'boost': 'gbdt',             'feature_fraction': 1,     'learning_rate': 0.0085,
    'max_depth': -1,             'metric':'auc',                'min_data_in_leaf': 80,     'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,            'num_threads': 8,              'tree_learner': 'serial',   'objective': 'binary',
    'reg_alpha': 0.1302650970728192, 'reg_lambda': 0.3603427518866501,'verbosity': 1
}


In [ ]:
folds = StratifiedKFold(n_splits=12, shuffle=False, random_state=99999)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    clf = lgb.train(param, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 2000)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = predictions
sub.to_csv("submission.csv", index=False)

Fold 0
Training until validation scores don't improve for 2000 rounds.
[1000]	training's auc: 0.88741	valid_1's auc: 0.862374
[2000]	training's auc: 0.917493	valid_1's auc: 0.887233
[3000]	training's auc: 0.931106	valid_1's auc: 0.898341
[4000]	training's auc: 0.939468	valid_1's auc: 0.903869
[5000]	training's auc: 0.945609	valid_1's auc: 0.906786
[6000]	training's auc: 0.950888	valid_1's auc: 0.908236
[7000]	training's auc: 0.9557	valid_1's auc: 0.908527
[8000]	training's auc: 0.960052	valid_1's auc: 0.908935
[9000]	training's auc: 0.964209	valid_1's auc: 0.908606
[10000]	training's auc: 0.968123	valid_1's auc: 0.908798
Early stopping, best iteration is:
[8055]	training's auc: 0.960264	valid_1's auc: 0.908967
Fold 1
